##***creating catalog***



In [0]:
%sql

create catalog ansh_lamba;


In [0]:
%sql
-- Set the catalog context
USE CATALOG ansh_lamba;

-- Create schema under the specified catalog
CREATE SCHEMA salesdata;

In [0]:
%sql
show schemas;

In [0]:
%sql
--  creating managed table

create table salesdata.sales_data (id int, name varchar(10))